In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path
from neo4j import GraphDatabase

In [2]:
load_dotenv(dotenv_path=Path("../.env"))

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [13]:
database = "neo4j"

In [5]:
def clear_db_tx(tx):
    # delete all nodes with relationships
    tx.run("""
        MATCH (a) -[r] -> () DELETE a, r
    """) 
    # delete nodes that have no relationships
    tx.run("""
        MATCH (a) DELETE a
    """)

def clear_db():
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session(database=database) as session:
            session.execute_write(clear_db_tx)
            print("cleared all nodes in " + database)

In [23]:
clear_db()

cleared all nodes in neo4j


In [21]:
def execute_write_tx(tx, cypher):
    tx.run(cypher) 

def execute_write(cypher):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        with driver.session(database=database) as session:
            session.execute_write(execute_write_tx, cypher)
            print("successful")

In [24]:
execute_write("""
CREATE (a:FirstNode { Tag : "This is a Node" })
    """)

successful


In [34]:
def execute_query(query: str, **kwargs: any):
    with GraphDatabase.driver(URI, auth=AUTH) as driver:
        records, summary, keys = driver.execute_query(
        query,
        **kwargs,
        database_=database,
        )
    return records, summary, keys

In [35]:
execute_query(
    """
MATCH (n:FirstNode) RETURN n LIMIT $limit
    """,
    limit = 25
)

([<Record n=<Node element_id='4:e2c4b0ec-a2c1-49c2-9dc4-b936b7c2ec5a:3' labels=frozenset({'FirstNode'}) properties={'Tag': 'This is a Node'}>>],
 ['n'])